In [1]:
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### 3 CVA/DVA Exercise

#### Market setup

##### Interest Rate Setup

In [ ]:
sigma_r = 0.02
c = 0.35
kappa1 = 0.02
kappa2 = 0.1
rho_inf = 0.4
nu = np.sqrt(1./c/c - 1. - 2.*(rho_inf/c - 1.))
rho_x = (rho_inf/c - 1.)*nu
sigma_l = c * sigma_r
sigma1 = sigma_l
sigma2 = nu*sigma1

###### Swap Object and Pricing

In [2]:
class Swap(object) :
    def __init__(self, maturity, coupon, freq, P_OIS, P_LIBOR) :
        '''
        Args:
            maturity: in years
            freq:     frequency of payments, 2 for semi-annual
            P_OIS:    array, discount bond price using OIS
            P_Libor:  array, discount bond price using LIBOR
        
        '''
        self.maturity = maturity
        self.coupon = coupon
        self.freq = freq
        '''self.P_OIS = P_OIS
        self.P_LIBOR = P_LIBOR'''
        
    def __str__(self) :
        return 'Swap: maturity %g, coupon %g, freq %g' % (self. maturity, self.coupon, self.freq)
    
def priceSwap(swap, payerOrReceiver, P_OIS, P_LIBOR, i, dt) :
    '''
    Args:
        swap:     a swap object
        payerOrReceiver: string type, 'payer' or 'receiver'
        P_OIS:    array, discount bond price using OIS
        P_Libor:  array, discount bond price using LIBOR
        i:        time step now, takes positive integer
        dt:       
    '''
    ts = np.arange(dt,swap.maturity+1e-6,dt)[i:]
    floating = 
    if payerOrReceiver == 'payer':
        net_payment = floating - swap.coupon
    else:
        net_payment = swap.coupon - floating
        
    return 

In [5]:
payer_swap = Swap(10,0.0265,2,1,1)
payer_swap.__str__()

'Swap: maturity 10, coupon 0.0265, freq 2'

###### Simulate OIS curve

In [ ]:
def simulateXt(rho_x, sigma1, sigma2, kappa1, kappa2, sim_freq, maturity):
    x = np.asarray([0,0])
    C = np.asarray([1,0],[rho_x,1-rho_x*rho_x]) # Cholesky decomposition of correlation matrix
    xt = []
    
    for i in range(maturity/sim_freq):
        wt = C.dot(np.random.normal(0,sim_freq,(2,)))
        y = 
        x = x + (y.dot(np.ones(2))-np.diag([kappa1,kappa2]).dot(x))*sim_freq+np.diag([sigma1,sigma2]).dot(wt)
        xt.append(x)

In [21]:
# test cell
rho_x = 0.5
C = np.asarray([[1,0],[rho_x,1-rho_x*rho_x]]) # Cholesky decomposition of correlation matrix
wt = C.dot(np.random.normal(0,1,(2,)))
print wt.shape
print np.ones(2).shape
print np.diag(np.asarray([2,3]))

(2L,)
(2L,)
[[2 0]
 [0 3]]


### 4 IMM Exercise